In [ ]:
%matplotlib inline

import numpy as np
from argparse import Namespace
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA
import phate

# Prepared embedded data

In [ ]:
"""load saved trajectories data for npz file
"""
SEQ = "PT4"
# SEQ = "PT3_hairpin"

# laod pre-training data
fnpz_data = "./data/pretraining/pretraining_{}.npz".format(SEQ)
data_npz = np.load(fnpz_data)

# asssign data to variables
for var in data_npz.files:
     locals()[var] = data_npz[var]
     print(var, locals()[var].shape)

In [ ]:
X_j = np.array(np.load(f'./data/graph/{SEQ}/shortestpath_knn=100.npz',allow_pickle=True)["X_j"], dtype=int)
D_ij = np.array(np.load(f'./data/graph/{SEQ}/shortestpath_knn=100.npz',allow_pickle=True)["D_ij"], dtype=float)
print("X_j", X_j.shape)
print("D_ij", D_ij.shape)

In [ ]:
# calculate the probability of being visited during a simulated trajectory 
# from the initial state
split_id = trj_id + 1 # index for split to each trajectory
P_tot = np.zeros(len(SIMS_dict_uniq))

for i in range(len(split_id)):
    if i == 0:
        trj = set(SIMS_dict[0:split_id[i],4].astype(int))
    else:
        trj = set(SIMS_dict[split_id[i-1]:split_id[i],4].astype(int))

    P_tot[list(trj)] += 1

P_tot = P_tot / 100

print("P_tot", P_tot.shape, P_tot.max(), P_tot.min())

In [ ]:
# load embedding WITHOUT vida plot data
# SEQ = "PT0"
fnpz_noViDa = f"./data/vida_data/noViDa-noEnergy/{SEQ}_noViDa.npz"
data_noViDa = np.load(fnpz_noViDa,allow_pickle=True)
for var in data_noViDa.files:
    globals()[var] = data_noViDa[var]
    print(var, globals()[var].shape)

In [ ]:
# load ViDa embedding plot data

fnpz_data_embed = f"./data/vida_data/{SEQ}.npz"
# fnpz_data_embed = f"./data/vida_data/{SEQ}_usePT4_03040216.npz"
data_npz_embed = np.load(fnpz_data_embed,allow_pickle=True)

# asssign data to variables
for var in data_npz_embed.files:
    globals()[var] = data_npz_embed[var]
    print(var, globals()[var].shape)
print("data_embed", data_embed.max(), data_embed.min(), data_embed.mean(), data_embed.std())

In [ ]:
# load GSAE embedding plot data

fnpz_data_embed = f"./data/gsae_data/helix_assoc_{SEQ}.npz"

data_npz_embed = np.load(fnpz_data_embed,allow_pickle=True)
# asssign data to variables
for var in data_npz_embed.files:
    globals()[var] = data_npz_embed[var]
    print(var, globals()[var].shape)
print("data_embed", data_embed.max(), data_embed.min(), data_embed.mean(), data_embed.std())

# Evaluate embedding

### Smoothness

In [ ]:
def get_trajec_embed(trj_id, embedding):
    trajec_embed = []
    TRJ_ID = trj_id+1
    
    for i in range(len(TRJ_ID)):
        if i == 0:
            s = 0
            s_prime = TRJ_ID[i]
        elif i == len(trj_id):
            s = TRJ_ID[i-1]
            s_prime = len(embedding)
        else:
            s = TRJ_ID[i-1]
            s_prime = TRJ_ID[i]
        
        trajec_embed.append(embedding[s:s_prime][:,:2])
    
    return trajec_embed

In [ ]:
def calculate_1trj_distance(arr):
    total_distance = 0
    
    for i in range(1, len(arr)):
        total_distance += np.sqrt(np.sum((arr[i] - arr[i-1])**2))
        
    return total_distance

In [ ]:
# calculate the expected total distance over all trajectories
def calculate_avg_total_distance(embedding, trj_id):
    scaler = MinMaxScaler()
    std_embedding = scaler.fit_transform(embedding) 
    
    trajec_embed = get_trajec_embed(trj_id, std_embedding)
    total_distance = 0
    
    for i in range(len(trajec_embed)):
        total_distance += calculate_1trj_distance(trajec_embed[i])        
        
    return total_distance / (trj_id[-1]+1)

In [ ]:
print(f"ViDa PCA: {calculate_avg_total_distance(pca_all_coords, trj_id):.3f}")
print(f"ViDa PHATE: {calculate_avg_total_distance(phate_all_coords, trj_id):.3f}")

In [ ]:
print(f"GSAE PCA: {calculate_avg_total_distance(pca_all_coords, trj_id):.3f}")
print(f"GSAE PHATE: {calculate_avg_total_distance(phate_all_coords, trj_id):.3f}")

In [ ]:
print(f"Direct PCA: {calculate_avg_total_distance(pca_all_coords_direct, trj_id):.3f}")
print(f"Direct PHATE: {calculate_avg_total_distance(phate_all_coords_direct, trj_id):.3f}")
# print(f"Direct MDS: {calculate_avg_total_distance(mds_all_coords, trj_id):.3f}")

### PDF

In [ ]:
def calculate_step_length(trj_id, embedding):
    trajec_embed = get_trajec_embed(trj_id, embedding)
    step_length = []
    
    for  i in range(len(trajec_embed)):
        arr = trajec_embed[i]
        for j in range(1, len(arr)):
            step_length.append(np.sqrt(np.sum((arr[j] - arr[j-1])**2)))
    return np.array(step_length)
            

In [ ]:
# def make_pdf(steps, num_bins):
#     # Sort the data
#     data = np.sort(steps)
    
#     # Group the data into the bins   
#     counts, bin_edges = np.histogram(data, bins=num_bins)
    
#     # Calculate the density of data points in each bin
#     density = counts / sum(counts)

#     # Plot the PDF
#     bin_widths = np.diff(bin_edges)
#     bin_centers = bin_edges[:-1] + bin_widths / 2
#     bin_centers = bin_centers / bin_centers.max() * 100

#     plt.plot(bin_centers, density)
#     plt.xticks(np.arange(0, 101, 10))
#     plt.xlabel('Step length percentage (%)')
#     plt.ylabel('Density')
#     plt.title('PDF of step length for embedding')
#     return plt.show()


In [ ]:
def make_pdf(steps, num_bins):
    # Normalize the data
    scaler = MinMaxScaler()
    norm_steps = scaler.fit_transform(steps.reshape(-1,1))
    
    # Sort the data
    data = np.sort(norm_steps)
    
    # Group the data into the bins   
    make_pdf.counts, make_pdf.bin_edges = np.histogram(data, bins=num_bins)
    
    # Calculate the density of data points in each bin
    make_pdf.density = make_pdf.counts / sum(make_pdf.counts)

    # Plot the PDF
    bin_widths = np.diff(make_pdf.bin_edges)
    make_pdf.bin_centers = make_pdf.bin_edges[:-1] + bin_widths / 2
    make_pdf.bin_centers = make_pdf.bin_centers / make_pdf.bin_centers.max() * 100

    plt.plot(make_pdf.bin_centers, make_pdf.density)
    plt.xticks(np.arange(0, 101, 10))
    plt.xlabel('Step length percentage (%)')
    plt.ylabel('Density')
    plt.title('PDF of step length for embedding')
    return plt.show()


In [ ]:
steps = calculate_step_length(trj_id, phate_all_coords)
# steps = calculate_step_length(trj_id, phate_all_coords_direct)

make_pdf(steps, num_bins=10)

In [ ]:
make_pdf.bin_edges, make_pdf.counts

In [ ]:
gsae_phate = make_pdf.counts
vida_pca, vida_phate, direct_pca, direct_phate, gsae_pca, gsae_phate

In [ ]:
vida_pca, vida_phate, direct_pca, direct_phate, gsae_pca, gsae_phate

In [ ]:
bin_widths = np.diff(make_pdf.bin_edges)
bin_centers = make_pdf.bin_edges[:-1] + bin_widths / 2
bin_centers = make_pdf.bin_centers / make_pdf.bin_centers.max() * 100
bin_centers

In [ ]:
# save for plotting
fnpz_data = f"data/vida_data/pdf_plot_{SEQ}.npz"
with open(fnpz_data, 'wb') as f:
    np.savez(f,
            bin_centers=bin_centers,
            vida_pca=vida_pca,
            vida_phate=vida_phate,
            gsae_pca=gsae_pca,
            gsae_phate=gsae_phate,
            direct_pca=direct_pca,
            direct_phate=direct_phate,
            # direct_mds=direct_mds,
            )

In [ ]:
## Make a whole plot

# Create a figure and axis object
fig, ax = plt.subplots(figsize =(15, 8))

# Plot the lines
ax.plot(bin_centers, vida_pca/sum(vida_pca), label='ViDa PCA')
ax.plot(bin_centers, vida_phate/sum(vida_phate), label='ViDa PHATE')
ax.plot(bin_centers, gsae_pca/sum(gsae_pca), label='GSAE PCA')
ax.plot(bin_centers, gsae_phate/sum(gsae_phate), label='GSAE PHATE')
ax.plot(bin_centers, direct_pca/sum(direct_pca), label='Direct PCA')
ax.plot(bin_centers, direct_phate/sum(direct_phate), label='Direct PHATE')
# ax.plot(bin_centers, direct_mds/sum(direct_mds), label='Direct MDS')

# Set the legend
ax.legend()

plt.xticks(np.arange(0, 101, 10))
plt.xlabel('Step length percentage (%)')
plt.ylabel('Density')
plt.title(f'PDF of step length for {SEQ}')

plt.show()

In [ ]:
# steps = calculate_step_length(trj_id, pca_all_coords_direct)
steps = calculate_step_length(trj_id, pca_all_coords)

make_pdf(steps, num_bins=1000)

### Metric distance

In [ ]:
def metric_dist(X_j, D_ij, P_tot, z):
    """
    Metric to calculate the distance 
    """
    z_re = z.reshape(-1,1,z.shape[-1])
    zj = z[X_j]
    global l2_zizj
    l2_zizj = np.sqrt(np.sum((z_re-zj)**2, axis=-1))
    
    # # normalize the distance
    # scaler = MinMaxScaler(feature_range=(0,1)) 
    # l2_zizj = scaler.fit_transform(l2_zizj)
    # D_ij = scaler.fit_transform(D_ij)
    
    dist_diff = (l2_zizj - D_ij)**2
    wij = (P_tot.reshape(-1,1) * P_tot[X_j])
    # dist_loss = np.sum(wij * dist_diff)
    dist_loss = ((wij * dist_diff) * P_tot.reshape(-1,1) / P_tot.sum()).sum()
    
    
    return dist_loss

In [ ]:
log_Dij = np.log(D_ij)+np.abs(np.log(D_ij).min())
print("log_Dij", log_Dij.shape, log_Dij.max(), log_Dij.min())

pca_dist = metric_dist(X_j, log_Dij, P_tot, pca_coords[:,:2])
phate_dist = metric_dist(X_j, log_Dij, P_tot, phate_coords)
print (f'ViDa PCA distance loss: {pca_dist:.4f}')
print (f'ViDa PHATE distance loss: {phate_dist:.4f}')

In [ ]:
log_Dij = np.log(D_ij)+np.abs(np.log(D_ij).min())

pca_dist = metric_dist(X_j, log_Dij, P_tot, pca_coords_direct[:,:2])
phate_dist = metric_dist(X_j, log_Dij, P_tot, phate_coords_direct)
print (f'Direct PCA distance loss: {pca_dist:.4f}')
print (f'Direct PHATE distance loss: {phate_dist:.4f}')

In [ ]:
log_Dij = np.log(D_ij)+np.abs(np.log(D_ij).min())

pca_dist = metric_dist(X_j, log_Dij, P_tot, pca_coords[:,:2])
phate_dist = metric_dist(X_j, log_Dij, P_tot, pca_coords)
print (f'GSAE PCA distance loss: {pca_dist:.4f}')
print (f'GSAE Direct PHATE distance loss: {phate_dist:.4f}')

### Neighboring preservation rate

In [ ]:
from sklearn.neighbors import NearestNeighbors

def neighboring_preservation_rate(X, X_j, P_tot, k):
    """
    Metric to calculate the neighboring preservation rate 
    """
    # Compute the k-nearest neighbors for both X and Y.
    nn_X = NearestNeighbors(n_neighbors=k+1).fit(X) # k+1 because we don't want to include the point itself
    indices_X = nn_X.kneighbors(X,return_distance=False)[:,1:] # exclude the point itself
    
    # compute the rate of each point
    rate_list = []
    for i in range(len(indices_X)):
        count = len(np.intersect1d(indices_X[i], X_j[i,:k]))
        rate_i = count/k
        rate_list.append(rate_i)
    
    # Compute the overall neighbsoring preservation rate
    
    # rate = np.mean(rate_list) # average
    rate = (rate_list * P_tot / P_tot.sum()).sum() # weighted average
    
    return rate

In [ ]:
# ViDa embedding
knn = 10000
print("ViDa PCA rate: {:.4f}".format(neighboring_preservation_rate(pca_coords[:,:2], X_j_all, P_tot, k=knn)))
print("ViDa PHATE rate: {:.4f}".format(neighboring_preservation_rate(phate_coords, X_j_all, P_tot, k=knn)))

In [ ]:
# Pretrained ViDa embedding
knn = 10000
print("Pretrained ViDa PCA rate: {:.4f}".format(neighboring_preservation_rate(pca_coords[:,:2], X_j_all, P_tot, k=knn)))
print("Pretrained ViDa PHATE rate: {:.4f}".format(neighboring_preservation_rate(phate_coords, X_j_all, P_tot, k=knn)))

In [ ]:
# GSAE embedding
knn = 10000
print("GSAE PCA rate: {:.4f}".format(neighboring_preservation_rate(pca_coords[:,:2], X_j_all, P_tot, k=knn)))
print("GSAE PHATE rate: {:.4f}".format(neighboring_preservation_rate(phate_coords, X_j_all, P_tot, k=knn)))

In [ ]:
# Direct embedding
knn = 100
print("Direct PCA rate: {:.4f}".format(neighboring_preservation_rate(pca_coords_direct[:,:2], X_j, P_tot, k=knn)))
print("Direct PHATE rate: {:.4f}".format(neighboring_preservation_rate(phate_coords_direct, X_j, P_tot, k=knn)))

In [ ]:
# D_ij_all = []
# X_j_all = []

# for i in range(len(SIMS_HT_uniq)):
#     dij = np.array(list(np.load(f'./data/graph/{SEQ}/allpath_{SEQ}/path_{i}.npy',allow_pickle=True)[0].values()), dtype=float)
#     xj = np.array(list(np.load(f'./data/graph/{SEQ}/allpath_{SEQ}/path_{i}.npy',allow_pickle=True)[0].keys()), dtype=int)
    
#     D_ij_all.append(dij)
#     X_j_all.append(xj)

# D_ij_all = np.stack(D_ij_all)
# X_j_all = np.stack(X_j_all)

# # save npz file for shortest path
# with open(f'./data/graph/{SEQ}_allpath.npz', 'wb') as f:
#     np.savez(f,
#              X_j_all = np.stack(X_j_all),
#              D_ij_all = np.stack(D_ij_all),
#          )

### PCA explained variance

In [ ]:
cm = PCA(n_components=25)
cm.fit(data_embed)

PC_values = np.arange(cm.n_components_) + 1
plt.plot(PC_values, np.cumsum(cm.explained_variance_ratio_), 'ro-', linewidth=2)
plt.title('Scree Plot: PCA')
plt.xlabel('Number of principal components')
plt.ylabel('Cumulative explained variance');
# plt.xticks(np.arange(0, data_embed.shape[-1]+1, 1))
plt.show()

print(np.cumsum(cm.explained_variance_ratio_))

### MDS for distance matrix

In [ ]:
# PT4_hairpin all path
X_j_all = np.array(np.load(f'./data/graph/{SEQ}_allpath.npz',allow_pickle=True)["X_j_all"])
D_ij_all = np.array(np.load(f'./data/graph/{SEQ}_allpath.npz',allow_pickle=True)["D_ij_all"])
print("X_j_all", X_j_all.shape)
print("D_ij_all", D_ij_all.shape)

In [ ]:
# make precomputed distance matrix for MDS with 
MDS_dist = np.ones((D_ij_all.shape[0],D_ij_all.shape[0]))
for i in range(len(D_ij_all)):
    MDS_dist[i,X_j_all[i]] = D_ij_all[i]

In [ ]:
MDS_dist.shape

In [ ]:
# not necessary if the distance matrix includes all the neighbors
def makeSymmetric(mat):
    # Loop to traverse lower triangular
    # elements of the given matrix
    for i in range(0, len(mat)):
        for j in range(0, len(mat)):
            if (j < i):
                mat[i][j] = mat[j][i] = min(mat[i][j], mat[j][i])
    return mat

In [ ]:
MDS_dist_symm = makeSymmetric(MDS_dist)
MDS_dist_symm.shape

In [ ]:
# %%script false --no-raise-error

from sklearn.manifold import MDS
mds = MDS(n_components=2, dissimilarity='precomputed')
mds_coords = mds.fit_transform(MDS_dist)

In [ ]:
# # save for python
# pca_all_coords_direct = pca_coords_direct[coord_id_S]  
# phate_all_coords_direct = phate_coords_direct[coord_id_S]
# mds_all_coords = mds_coords[coord_id_S]

# fnpz_data = f"data/vida_data/noViDa-noEnergy/{SEQ}_noViDa.npz"
# with open(fnpz_data, 'wb') as f:
#     np.savez(f,
#             pca_coords_direct=pca_coords_direct, pca_all_coords_direct=pca_all_coords_direct,
#             phate_coords_direct=phate_coords_direct, phate_all_coords_direct=phate_all_coords_direct,
#             mds_coords=mds_coords, mds_all_coords=mds_all_coords,
#             )

# Visualize

In [ ]:
SEQ

### PCA Vis

In [ ]:
%matplotlib inline
X = pca_coords[:,0]
Y = pca_coords[:,1]
Z = pca_coords[:,2]

# PCA: 2 components
fig,ax = plt.subplots(figsize=(8,6))
im = ax.scatter(X, Y, 
          c=SIMS_G_uniq, 
          cmap='plasma',
        )

plt.colorbar(im)

annotations=["I","F"]
x = [X[0],X[int(SIMS_dict[-1,-1])]]
y = [Y[0],Y[int(SIMS_dict[-1,-1])]]
plt.scatter(x,y,s=150, c="green", alpha=1)
for i, label in enumerate(annotations):
    plt.annotate(label, (x[i],y[i]),fontsize=20,c="black")

#### Try use PCA directly without AE

In [ ]:
# %%script false --no-raise-error

pca_coords_direct = PCA(n_components=3).fit_transform(SIMS_scar_uniq)   # multiple trj

X = pca_coords_direct[:,0]
Y = pca_coords_direct[:,1]
Z = pca_coords_direct[:,2]

# PCA: 2 components
fig,ax = plt.subplots(figsize=(8,6))
im = ax.scatter(X, Y, 
          c=SIMS_G_uniq, 
          cmap='plasma',
        )

plt.colorbar(im)

annotations=["I","F"]
x = [X[0],X[-1]]
y = [Y[0],Y[-1]]
plt.scatter(x,y,s=150, c="green", alpha=1)
for i, label in enumerate(annotations):
    plt.annotate(label, (x[i]-0.3,y[i]-0.3),fontsize=20,c="black")

In [ ]:
cm = PCA(n_components=25)
cm.fit(SIMS_scar_uniq)

PC_values = np.arange(cm.n_components_) + 1
plt.plot(PC_values, np.cumsum(cm.explained_variance_ratio_), 'ro-', linewidth=2)
plt.title('Scree Plot: PCA')
plt.xlabel('Number of principal components')
plt.ylabel('Cumulative explained variance');
# plt.xticks(np.arange(0, data_embed.shape[-1]+1, 1))
plt.show()

print(np.cumsum(cm.explained_variance_ratio_))

### PHATE Vis

In [ ]:
X_phate = phate_coords[:,0]
Y_phate = phate_coords[:,1]

fig,ax = plt.subplots(figsize=(8,6))
im = ax.scatter(X_phate,Y_phate,
                c=SIMS_G_uniq,            
                cmap='plasma',
               )

plt.colorbar(im)

annotations=["I","F"]
x = [X_phate[0],X_phate[int(SIMS_dict[-1,-1])]]
y = [Y_phate[0],Y_phate[int(SIMS_dict[-1,-1])]]
plt.scatter(x,y,s=50, c="green", alpha=1)
for i, label in enumerate(annotations):
    plt.annotate(label, (x[i],y[i]),fontsize=20,c="black")

#### PHATE without AE

In [ ]:
# %%script false --no-raise-error

phate_operator = phate.PHATE(n_jobs=-2)
phate_coords_direct = phate_operator.fit_transform(SIMS_scar_uniq)

fig,ax = plt.subplots(figsize=(8,6))
im = ax.scatter(phate_coords_direct[:,0],
          phate_coords_direct[:,1],
          c=SIMS_G_uniq, 
          cmap='plasma',
        )

plt.colorbar(im)

annotations=["I","F"]
x = [phate_coords_direct[:,0][0],phate_coords_direct[:,0][-1]]
y = [phate_coords_direct[:,1][0],phate_coords_direct[:,1][-1]]
plt.scatter(x,y,s=50, c="green", alpha=1)
for i, label in enumerate(annotations):
    plt.annotate(label, (x[i],y[i]),fontsize=20,c="black")

### MDS with distance matrix

In [ ]:
X = mds_coords[:,0]
Y = mds_coords[:,1]
cmap = plt.cm.plasma
cmap_r = plt.cm.get_cmap('plasma_r')

fig,ax = plt.subplots(figsize=(8,6))
im = ax.scatter(X, Y, 
          c = SIMS_G_uniq,
          cmap=cmap,
          s=10
        )
 
plt.colorbar(im)

annotations=["I","F"]
x = [X[0],X[int(SIMS_dict[-1,-1])]]
y = [Y[0],Y[int(SIMS_dict[-1,-1])]]
plt.scatter(x,y,s=150, c="green", alpha=1)
for i, label in enumerate(annotations):
    plt.annotate(label, (x[i],y[i]),fontsize=20,c="black")

In [ ]:
pca_coords.min(), pca_coords.max()

In [ ]:
phate_coords.min(), phate_coords.max(), mds_coords.min(), mds_coords.max()